<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/SeriesdeTiempo/EMAE_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
from google.colab import userdata

# Definir user key para la API
user_key = userdata.get('ALPHACAST')

In [3]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [4]:
emae.set_index('Date', inplace = True)
emae = emae.loc['2016-02-01':]
y=emae['Emae - current_prices_mom']

In [6]:
# IPI
ipi = requests.get('https://api.alphacast.io/datasets/5679/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('ipi.csv','wb').write(ipi.content)
ipi = pd.read_csv('ipi.csv',delimiter = ',')


ipi['Date'] = pd.to_datetime(ipi['Date'])
ipi.set_index('Date', inplace = True)

In [9]:
ipi.head()

,Country,Division,Seasonal Adjustment,Subclass,Value,Value - current_prices_3m_yoy,Value - current_prices_yoy,Value - current_prices_mom
Date,,,,,,,,
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Azúcar, productos de confitería y chocolate",50.680600,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne aviar,212.009238,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne vacuna,83.169668,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Fiambres y embutidos,143.350871,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Galletitas, productos de panadería y pastas",95.801964,NaN,NaN,NaN


In [10]:
ipi = ipi[(ipi['Division'] == "IPI Manufacturero") & (ipi['Seasonal Adjustment'] == "Seasonally Adjusted")]
ipi = ipi['Value']
ipi.columns = ['Value - sa_MoM']
ipi = ipi.loc['2016-02-01':]

In [11]:
# ISAC
isac = requests.get('https://api.alphacast.io/datasets/5565/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('isac.csv','wb').write(isac.content)
isac = pd.read_csv('isac.csv',delimiter = ',')


isac['Date'] = pd.to_datetime(isac['Date'])
isac.set_index('Date', inplace = True)
isac = isac['Nivel general - sa_MoM']
isac.columns = ['Nivel general - sa_MoM']
isac = isac.loc['2016-02-01':]

In [12]:
isac=pd.Series(isac, name='ISAC')
ipi=pd.Series(ipi, name='IPI')

In [13]:
X = pd.concat([ipi, isac], axis=1)

In [15]:
y.shape

(101,)

In [16]:
X.shape

(101, 2)

In [17]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X[:-1], y[:-1]) #Si ya salió IPI e ISAC, pero todavía no el EMAE entonces hay que fitear lr.fit(X[:-1], y)
Yest = lr.predict(X)

In [18]:
Yest

array([ 0.70707339, -0.21560035, -0.51018842,  0.63298377, -0.21824554,
       -0.08287598,  1.41190761,  0.42925352, -1.36074656,  2.19039298,
        0.85902215,  0.3802231 , -0.12126329,  1.74594077, -0.56216148,
        0.96048376,  1.18286509,  0.18223498,  0.88243525,  0.5659066 ,
        0.89891726,  1.40893016, -0.02496963,  0.79159428, -0.07549551,
        0.71031013,  0.17195615,  0.02290643,  0.01979046, -0.1770622 ,
        0.47182811, -0.27655272,  0.04800024, -0.9648547 , -0.76892509,
        1.08963751,  1.33023711, -1.04685215,  0.53401441,  0.34849742,
       -1.32026702,  1.04080565, -0.33954747, -0.7666764 , -0.06784784,
       -0.4544313 , -0.74308697, -0.14884686,  0.11030349, -6.34152966,
       -7.88624515, 12.82230363,  6.96137277,  0.84557037, -1.38108585,
        1.82189878,  0.40204812,  0.72425145,  1.84227751, -0.23531382,
       -0.14678642,  0.67535357, -0.28432903, -1.3802181 ,  2.14012688,
        0.24557414, -0.19745448,  0.55496306,  0.10274888,  0.82

In [20]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'X + ', lr.intercept_)
print()
print('Precisión del modelo:')
print(lr.score(X[:-1], y[:-1]))

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[0.02970863 0.14391169]
Valor de la intersección o coeficiente "b":
-3.4966567953028087

La ecuación del modelo es igual a:
y =  [0.02970863 0.14391169] X +  -3.4966567953028087

Precisión del modelo:
0.1442587699962925


In [21]:
lr.coef_[0]

0.029708627528708145

In [22]:
emae_predict=X[-1:]['IPI']*lr.coef_[0]+X[-1:]['ISAC']*lr.coef_[1]+lr.intercept_

In [23]:
emae_predict

,0
Date,
2024-06-01,-0.755273
